<a href="https://colab.research.google.com/github/toche7/PromptEngforDeveloper/blob/main/PromptLab4_inferring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferring
ใน notebook นี้เราจะได้ทดลองการใช้ LLM ในการอนุมาน หัวข้อ อารมณ์ และความรู้สึก จากการ review สินค้า และจากบทความข่าว

## Setup

In [ ]:
# !pip -q install python-dotenv
!pip -q install groq

In [ ]:
#=========VSCode=====================================================================
# from dotenv import find_dotenv, load_dotenv
# _ = load_dotenv(find_dotenv())
# import os
# secret_key = os.environ.get("GROQ_API_KEY")
#========google.colab======================= 
from google.colab import userdata
secret_key = userdata.get('GROQ_API_KEY')

#===== create Groq object================= 
from groq import Groq
clientGroq = Groq(
    api_key= secret_key,
)

In [ ]:
def get_completion(prompt, model= "llama3-70b-8192", clientGroq = clientGroq):
    messages = [{"role": "user", "content": prompt}]
    response = clientGroq.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        max_tokens= 4000,
        top_p = 0.5
    )
    return response.choices[0].message.content

## Product review text

In [ ]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [ ]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple ticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple ticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## Identify types of emotions

In [ ]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## Identify anger

In [ ]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## Extract product and company name from customer reviews

In [ ]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## Doing multiple tasks at once

In [ ]:
prompt = f"""
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## Inferring topics

In [ ]:
story = """
In a recent survey conducted by the government,
public sector employees were asked to rate their level
of satisfaction with the department they work at.
The results revealed that NASA was the most popular
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings,
stating, "I'm not surprised that NASA came out on top.
It's a great place to work with amazing people and
incredible opportunities. I'm proud to be a part of
such an innovative organization."

The results were also welcomed by NASA's management team,
with Director Tom Johnson stating, "We are thrilled to
hear that our employees are satisfied with their work at NASA.
We have a talented and dedicated team who work tirelessly
to achieve our goals, and it's fantastic to see that their
hard work is paying off."

The survey also revealed that the
Social Security Administration had the lowest satisfaction
rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to
address the concerns raised by employees in the survey and
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics

In [ ]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple ticks.

Make each item one or two words long.

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
topic_list = [
    "nasa", "local government", "engineering",
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [ ]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\
for example
nasa : 1

List of topics: {", ".join(topic_list)}

Format your response as a JSON object.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
import json
def extract_json_part(text):
  # Find the start and end of the JSON object
  start = text.find('{')
  end = text.rfind('}') + 1
  
  # Extract the JSON object as a string
  json_string = text[start:end]
  
  # Attempt to parse the JSON string
  try:
    json_object = json.loads(json_string)
    return json_object
  except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    return None


In [ ]:
extracted_json = extract_json_part(response)
print(extracted_json)

In [ ]:
if extracted_json['nasa'] == 1:
    print("ALERT: New 

## Try experimenting on your own!